## Cluster TMAs

**Samples:** 
- U54 TMA MIBI

**Method**: 


- leiden cluster on good biomarkers for celltypes


In [ ]:
#load libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import scipy
import scanpy as sc
import matplotlib as mpl
from matplotlib import cm
import math
mpl.rc('figure', max_open_warning = 0)
codedir = '/home/groups/graylab_share/Chin_Lab/ChinData/engje/Data/20200000/20200406_JP-TMAs'#os.getcwd()
import phenograph
os.chdir('/home/groups/graylab_share/Chin_Lab/ChinData/engje/Data')
from mplex_image import visualize as viz, process, preprocess, normalize, mics, mpimage
np.random.seed(126)

In [ ]:
#sorted(os.listdir(datadir))

In [ ]:
#change to correct directory
datadir = f'/home/groups/graylab_share/Chin_Lab/ChinData/engje/Data/20200000/20200406_JP-TMAs/data'

s_date = '20240520'
os.chdir(codedir)
if not os.path.exists(s_date):
    os.mkdir(s_date)
%matplotlib inline

# Table of contents <a name="contents"></a>
1. [load data](#cluster)
3. [select normal tissue](#load)
5. [single cell Umap](#l7)
6. [Leiden clustering](#l8)
7. [annotate leiden celltypes](#lbar)
7. [Tissue means](#tissue)


# Normal Tissues  <a name="cluster"></a>


select normal tissue from PDAC TMA (done)


[contents](#contents)

In [ ]:
#sorted(os.listdir(f'{datadir}'))

In [ ]:
# #MIBI int
# os.chdir(datadir)
# s_sample = 'cell_table_size_normalized_cell_labels.csv'
# df_mi = pd.read_csv(f'{datadir}/{s_sample}')
# df_mi.columns

In [ ]:
# #select

# ls_col=['label','fov','centroid-0', 'centroid-1','area', 'eccentricity',
#        'Amylase','BCAT', 'CD11c', 'CD14', 'CD163', 'CD20', 'CD206', 'CD209', 'CD3',
#        'CD31', 'CD4', 'CD44', 'CD45', 'CD56', 'CD68', 'CD74', 'CD8', 'CD9',
#        'CHP', 'COL1A1', 'Calprotectin', 'Chym_Tryp', 'ECAD', 'FAP', 'FN1',
#        'FOXP3', 'GATA6', 'HLADR', 'ITGB1', 'KRT17', 'KRT18', 'Ki67', 'MYC',
#        'Nuclear', 'PIN1', 'PanKRT', 'Podoplanin', 'SMA', 'Synaptophsyin',
#        'TCF7', 'Vimentin',]
# df_mi=df_mi.loc[:,ls_col]
# df_mi.rename({ 'centroid-1':'DAPIX', 'centroid-0':'DAPIY',},axis=1,inplace=True)

In [ ]:
# #shorten and check
# df_mi.columns = [item.split('_')[0] for item in df_mi.columns]
# print(len(df_mi.columns))
# df_mi.columns[df_mi.columns.duplicated()]

In [ ]:
# #df_mi = df_mi.merge(df_xy,left_index=True, right_index=True)
# df_mi.rename({'DAPIX':'DAPI_X','DAPIY':'DAPI_Y'},axis=1,inplace=True)
# #df_mi['slide_scene'] = [item.split('_cell')[0] for item in df_mi.index]

In [ ]:
#read in metadata
df_meta = pd.read_csv(f'{datadir}/U54-TMA_annotation_update_18Oct2023.csv')
df_meta[df_meta.Tissue.str.contains('Normal')]

In [ ]:
# df_mi['slide_scene'] = df_mi.fov.map(dict(zip(df_meta.MIBI_fov,df_meta.coor_mplexable)))
# df_mi['Tissue'] = df_mi.fov.map(dict(zip(df_meta.MIBI_fov,df_meta.Tissue)))
# df_mi = df_mi[df_mi.Tissue.str.contains('Normal')]

In [ ]:
# df_mi.Tissue.unique()

In [ ]:
# fig, ax = plt.subplots()
# __ = sns.kdeplot(df_mi.ECAD,ax=ax
#            )
# ax.axvline(0.000005)
# #df_mi = df_mi[df_mi.DAPI11Q>1000]

In [ ]:
# #save
# #s_out = f'{codedir}/data/20211221_JP-TMA2_FilteredMeanIntensity.csv'
# #s_out = f'{codedir}/data/20240523_Her2B-K154-157_FilteredMeanIntensity.csv'
# s_out = f'{codedir}/data/20240528_U54-TMA-Normal-MIBI_FilteredMeanIntensity.csv'
# if not os.path.exists(s_out):
#     print('saving csv')
#     df_mi.to_csv(s_out)

In [ ]:
# df_mi['Ecad_Pos'] = df_mi.ECAD>0.00002

In [ ]:
# ## visualize


# def plot_pos(df_pos,ls_color,df_xy):
#     #plot
#     fig, ax = plt.subplots(figsize=(5,5)) #figsize=(18,12)
#     #plot negative cells
#     ax.scatter(data=df_xy,x='DAPI_X',y='DAPI_Y',color='silver',s=1)
#     for idx,s_color in enumerate(ls_color):
#         #positive cells = positive cells based on threshold
#         ls_pos_index = (df_pos[df_pos.loc[:,s_color]]).index
#         df_color_pos = df_xy[df_xy.index.isin(ls_pos_index)]
#         if len(df_color_pos)>=1:
#             #plot positive cells
#             ax.scatter(data=df_color_pos, x='DAPI_X',y='DAPI_Y',color=f'C{idx}',s=2,alpha=0.8)
#     ax.axis('equal')
#     ax.set_ylim(ax.get_ylim()[::-1])
#     ax.set_title(f'{s_color}')
#     fig.suptitle(f"{s_scene}\n{df_pos.loc[:,'Tissue'].iloc[0]}",y= 0.99)
#     return(fig, ax)


In [ ]:
# #plot ROIS
# for s_scene in df_mi.slide_scene.unique():
#     df_scene=df_mi[df_mi.slide_scene==s_scene]
#     fig, ax = plot_pos(df_scene,['Ecad_Pos'],df_scene) #,'ROI2','ROI3','ROI0','ROI1'
#     #break

## Umap <a name="l7"></a>

umap projection of single cells

first use all markers, all subtypes, just to evaluate markers

[contents](#contents)

In [ ]:
import scanpy as sc
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
os.chdir(datadir)
#load normal cores
s_out = f'{codedir}/data/20240528_U54-TMA-Normal-MIBI_FilteredMeanIntensity.csv'
df_norm  = pd.read_csv(f'{s_out}',index_col=0) 


In [ ]:
#Exclude bad cells 
df_exclude = pd.DataFrame()# df_norm[(df_norm.slide_scene=='U54-TMA-9_sceneF08') & (df_norm.DAPI_Y<400)]
ls_exclude = []
#ls_exclude_all = sorted(set(df_norm[df_norm.loc[:,'PD1'] > 1365].loc[:,'PD1'].index.tolist()).union(set(ls_exclude))) #PD1
#df_exclude = df_exclude.append(pd.DataFrame(index=ls_exclude_all,columns=df_exclude.columns))
df_exclude_er = df_exclude.copy()
print(len(df_exclude_er))

In [ ]:
#32 features
#Her2B-K154-6S_scene009',       'Her2B-K157-6S
s_sample = s_out.split('data/')[1].split('_Filtered')[0]
s_type = 'MIBI'#

#select cell type markers
ls_col = ['Amylase',
       'BCAT', 'CD11c', 'CD14', 'CD163', 'CD20', 'CD206',  'CD3','CD209',
       'CD31', 'CD4', 'CD45', 'CD56', 'CD68', 'CD74', 'CD8','CD9', #'CD44', 
        'COL1A1', 'ECAD', #'FAP', 'FN1',# 'Calprotectin', 'Chym_Tryp','CHP','FOXP3',
         'HLADR',  'KRT18', #'KRT17' not in normal tissue,'GATA6','Ki67', # 'ITGB1','MYC', 'PIN1','TCF7',
        'PanKRT',  'SMA', 'Synaptophsyin',#'Nuclear','Podoplanin',
        'Vimentin',]
print(len(ls_col))
adata = sc.AnnData(df_norm.loc[(~df_norm.index.isin(df_exclude.index)),ls_col]) 
adata.obs['slide_scene'] = adata.obs.index.map(dict(zip(df_norm.index.astype('str'),df_norm.slide_scene)))
adata.obs['subtype'] = adata.obs.index.map(dict(zip(df_norm.index.astype('str'),df_norm.Tissue)))
# #two subtypes
# if s_type == 'all':
#     adata = adata[adata.obs.subtype.isin(['Normal', 'Border', 'Tumor'])]
# else:
#     adata = adata[adata.obs.TMA.isin([s_type])]  
print(len(adata))

In [ ]:
#include two TNBC TMAs
#print(adata.obs['TMA'].unique())
print(adata.obs['subtype'].unique())

In [ ]:
#raw
b_scale = True
adata.raw = adata
#reduce dimensionality
sc.tl.pca(adata, svd_solver='auto')
fig,ax=plt.subplots(figsize=(3.5,5))
sc.pl.highest_expr_genes(adata, n_top=48,ax=ax,save=f'Expression_{len(ls_col)}.png')
plt.tight_layout()
sc.pl.pca_variance_ratio(adata,n_pcs=32, log=True)

if b_scale:
    sc.pp.scale(adata, zero_center=False, max_value=20)
    s_sample = s_sample + '_s'

In [ ]:
#umap
for n_neighbors in [30,15]:
    results_file = f'{s_sample}_{n_neighbors}neighbors_{s_type}_{len(ls_col)}markers.h5ad'
    if not os.path.exists(results_file):
        print('calc umap')
        # calculate neighbors 
        sc.pp.neighbors(adata, n_neighbors=n_neighbors) #, method='rapids'
        sc.tl.umap(adata)
    else:
        print('loading umap')
        adata = sc.read_h5ad(results_file)
        adata.obs['slide_scene'] = adata.obs.index.map(dict(zip(df_norm.index.astype('str'),df_norm.slide_scene)))
        adata.obs['subtype'] = adata.obs.index.map(dict(zip(df_norm.index.astype('str'),df_norm.Tissue)))
    #save results

    #color by markers
    #figname = f"Umap{s_sample}_markers_{n_neighbors}neighbors_{len(ls_col)}markers.png"
    #sc.pl.umap(adata, color=ls_col,vmin='p1.5',vmax='p98.5',save=figname,ncols=6)
    #color by TMA
    # figname = f"Umap{s_sample}_TMA_{n_neighbors}neighbors_{s_type}_{len(ls_col)}markers.png"
    # fig,ax = plt.subplots(figsize=(3,2), dpi=200)
    # sc.pl.umap(adata, color='TMA',wspace=.25,save=figname,ax=ax)
    
    figname = f"Umap{s_sample}_subtype_{n_neighbors}neighbors_{s_type}_{len(ls_col)}markers.png"
    fig,ax = plt.subplots(figsize=(3,2), dpi=200)
    sc.pl.umap(adata, color='subtype',wspace=.25,save=figname,ax=ax)
    
    if not os.path.exists(results_file):
        print("writing")
        adata.write(results_file)
    break


In [ ]:
figname = f"Umap{s_sample}_markers_{n_neighbors}neighbors_{len(ls_col)}markers.png"
sc.pl.umap(adata, color=ls_col,vmin='p1.5',vmax='p98.5',save=figname,ncols=6)

##  Cluster Leiden <a name="l8"></a>

cluster umap projection of single cells


[contents](#contents)

In [ ]:
#cluster
#s_sample = '20240521_NP-Tissues-ROIs_s'#'
#n_neighbors = 30 #
n_markers = len(ls_col) #19 #
#s_type = 'NP9-IDC2'#'NP8-IDC'#'all' #
resolution = 0.6
results_file = f'{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}_leiden{resolution}.h5ad'
if not os.path.exists(results_file):
    print('clustering')
    sc.tl.leiden(adata,resolution=resolution)
    adata.write(results_file)
else:
    print('loading leiden')
    adata = sc.read_h5ad(results_file)


figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}_{s_type}_nolegend.png'
sc.pl.umap(adata, color='leiden',ax=ax,save=figname,title=figname.split('.png')[0].replace('_',' '),legend_loc=None)

fig,ax = plt.subplots(figsize=(3,2),dpi=200)
sc.pl.umap(adata, color='leiden',ax=ax,title=s_type)

# figname = f"Umap{s_sample}_subtype_{n_neighbors}neighbors_{s_type}_{len(ls_col)}markers.png"
# fig,ax = plt.subplots(figsize=(3,2), dpi=200)
# sc.pl.umap(adata, color='subtype',wspace=.25,save=figname,ax=ax)

In [ ]:
# sc.pl.umap(adata, color='Vim',vmin='p1.5',vmax='p98.5')
# sc.pl.umap(adata, color='ColI',vmin='p1.5',vmax='p98.5')
# sc.pl.umap(adata, color='Ecad',vmin='p1.5',vmax='p98.5')
# sc.pl.umap(adata, color='Ecad',vmin='p1.5',vmax='p98.5')
figname = f"Umap{s_sample}_scene_{n_neighbors}neighbors_{s_type}_{len(ls_col)}markers.png"
fig,ax = plt.subplots(figsize=(3,2), dpi=200)
sc.pl.umap(adata, color='slide_scene',wspace=.25,save=figname,ax=ax)

In [ ]:
len(adata)

## Annotate Leiden <a name="lbar"></a>

annotate epitelial, immune a stroma cell types

also visualize cluster results on tissue scatter plots. Additional visualization done using 20201018_JP-TMAs_napari.py,
to visualize ome.tiff created in 20201005_JP-TMA_Pipeline.py


[contents](#contents)

In [ ]:
os.chdir(f'{datadir}')
datadir

In [ ]:
#load ROIS

s_out = f'{codedir}/data/20240528_U54-TMA-Normal-MIBI_FilteredMeanIntensity.csv'
df_norm  = pd.read_csv(f'{s_out}',index_col=0) 
d_tissue = dict(zip(df_norm.index.astype('str'),df_norm.Tissue))

In [ ]:
#load leiden
s_sample = s_out.split('_Filtered')[0].split('data/')[1] + '_s' #'20240521_NP-Tissues-ROIs_s'#''20240522_NP9-IDC2-ROIs_s'#
n_neighbors = 30 #
resolution = 0.6
s_type='MIBI'#'Her2B-K154-6S'#

if s_type == 'MIBI':
    n_markers = 25 #29
else:
    n_markers = 25 #
results_file = f'{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}_leiden{resolution}.h5ad'
adata = sc.read_h5ad(results_file)

In [ ]:
results_file
df_p.columns

In [ ]:
fig, ax = plt.subplots()
sns.kdeplot(df_p.loc[(df_p.leiden=='5'),'KRT18'],ax=ax)
ax.axvline(2)

In [ ]:
fig, ax = plt.subplots()
sns.kdeplot(df_p.loc[(df_p.leiden=='5'),'PanKRT'],ax=ax)
#ax.axvline(3)

In [ ]:
# annotate
df_p = adata.to_df()
df_p['leiden'] = adata.obs['leiden'].astype('str')
df_p['Tissue'] = df_p.index.map(d_tissue)
print(df_p.Tissue.unique())
if results_file =='20240528_U54-TMA-Normal-MIBI_s_30neighbors_25markers_MIBI_leiden0.6.h5ad':
    print(s_type)
    ls_drop = ['11b'] #14 miss segmented, 
    df_p.loc[(df_p.leiden=='11') & (df_p.Tissue!='Normal_Liver'),'leiden'] = '11b' 
    df_p.loc[(df_p.leiden=='9') & (df_p.Tissue!='Normal_Pancreas'),'leiden'] = '11b' 
    df_p.loc[((df_p.leiden=='5')) & (df_p.KRT18>1),'leiden'] = '12'
    df_p.loc[((df_p.leiden=='5')) & (df_p.PanKRT>0.5),'leiden'] = '12'
    tum_clust = ['12','11','9','5','3']
    str_clust = []
    imm_clust = ['13','6','7','4','1','2','0'] 
    endo_clust = ['8'] 
    fb_clust = ['10']
    d_named = {'13':'NK cell','12':'panCK+ ep.','11':'Liver sinus',
               '10':'ColI+ Fb.','9':'Acinar','8':'endothelial',
               '7':'B cell','6':'Dentritic cell',
               '5':'Epithelial','3':'Epithelial','4':'T cell',
               '1':'B cell','2':'B cell','0':'T cell',
               #'2':'Ag. pres. imm.'
              }

else:
    print('new one')
    ls_drop=[]
    tum_clust = []
    str_clust = [] 
    imm_clust = [] 
    endo_clust = []
    fb_clust = []

    
#df_p.groupby('leiden').mean().Ecad.sort_values(ascending=False).index
#set(df_p.groupby('leiden').mean().CK19.sort_values(ascending=False).index) - set(tum_clust + imm_clust )
#set(tum_clust)

In [ ]:
#sns.kdeplot(df_p.CD68)

In [ ]:
#clustermap
if s_type=='Her2B-K154-6S':
    ls_drop_col = []
else:
    ls_drop_col = ['Tissue']
b_annot = True#True#False
d_replace = {}
d_replace.update(dict(zip(tum_clust,[f'epithelial' for item in tum_clust])))
d_replace.update(dict(zip(str_clust,[f'stromal' for item in str_clust])))
d_replace.update(dict(zip(imm_clust,[f'immune' for item in imm_clust])))
d_replace.update(dict(zip(endo_clust,[f'endothelial' for item in endo_clust])))
d_replace.update(dict(zip(fb_clust,[f'fibroblast' for item in fb_clust])))
df_plot = df_p.loc[~df_p.leiden.isin(ls_drop),~df_p.columns.isin(ls_drop_col)].groupby('leiden').mean()

df_plot.index.name = f'leiden {resolution}'
d_color = dict(zip(['endothelial', 'epithelial', 'fibroblast', 'immune', 'stromal'],sns.color_palette()[0:5]))
if b_annot:
    row_colors = df_plot.index.astype('str').map(d_replace).map(d_color)
    g = sns.clustermap(df_plot,z_score=1,figsize=(9,7),cmap='viridis',row_colors=row_colors,#method='single',
                   vmin=-1.5,vmax=1.5)
    for label in d_color.keys():
        g.ax_row_dendrogram.bar(0, 0, color=d_color[label],
                                label=label, linewidth=0)
    g.ax_row_dendrogram.legend(loc="right", ncol=1)
    g.ax_heatmap.set_ylabel('')
    marker_genes = df_plot.iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
    categories_order = df_plot.iloc[g.dendrogram_row.reordered_ind,:].index.tolist()
    labels = [f"{item}: {d_named[item]}" for item in categories_order]
    g.ax_heatmap.set_yticklabels(labels,rotation=0)
else:
    g = sns.clustermap(df_plot,z_score=1,figsize=(9,7),cmap='viridis',#row_colors=row_colors,#method='single',
                   vmin=-1.5,vmax=1.5)
    marker_genes = df_plot.iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
    categories_order = df_plot.iloc[g.dendrogram_row.reordered_ind,:].index.tolist()
g.fig.suptitle(s_type)
g.savefig(f'{codedir}/{s_date}/{s_sample}_clustermap_{n_neighbors}_{n_markers}markers_{s_type}_leiden{resolution}.png',dpi=200)


In [ ]:
fig,ax=plt.subplots(figsize=(1.4,3.7),dpi=200)
df_p.groupby('leiden').count().loc[categories_order[::-1]].iloc[:,1].plot(kind='barh',title='Cell No.',ax=ax,width=0.7)
plt.tight_layout()
fig.savefig(f'{codedir}/{s_date}/barplot_{s_sample}_{n_neighbors}neighbors_{len(marker_genes)}markers_leiden{resolution}_{s_type}.png')

In [ ]:
from mplex_image import analyze
df_data = df_norm
colors = mpl.cm.tab10.colors + mpl.cm.Accent.colors + mpl.cm.tab20b.colors + mpl.cm.tab20c.colors

df_pos = analyze.celltype_to_bool(df_p,'leiden')
df_pos.columns = [str(item) for item in df_pos.columns]

In [ ]:

s_clust = '12'
df_p['slide_scene'] = df_p.index.map(dict(zip(df_norm.index.astype('str'),df_norm.slide_scene)))#[item.split('_cell')[0] for item in df_p.index]

ls_sene = df_p.groupby(['leiden','slide_scene']).count().loc[s_clust].iloc[:,1].sort_values(ascending=False).index[0:5]
df_p.groupby(['leiden','slide_scene']).count().loc[s_clust].iloc[:,1].sort_values(ascending=False)[0:30]

In [ ]:
#plot all groups spatially - leiden 
ls_clust = df_pos.columns.tolist()#[s_clust,'1']##['8','11','12','7','5','10','1','9']#'0','2',
for s_slide in ls_sene:#sorted(set(df_p.slide_scene)):
    s_slide = 'U54-TMA-9_sceneA01'
    fig,ax = plt.subplots(figsize=(5,4.5),dpi=200)
    #plot negative cells
    df_scene = df_data[df_data.slide_scene.str.contains(s_slide)].copy()
    df_scene.index = df_scene.index.astype('str')
    ax.scatter(data=df_scene,x='DAPI_X',y='DAPI_Y',color='silver',s=1,label=f'')
    #for idxs, s_color_int in enumerate(range(len(df_pos.columns))):
    for idxs, s_color in enumerate(ls_clust): #enumerate(df_pos.columns):
        #s_color = s_clust
        s_color = str(s_color)
        if len(df_p[(df_p.slide_scene==s_slide) & (df_pos.loc[:,s_color])])>=1:
            #plot positive cells
            ls_index = df_p[(df_p.slide_scene==s_slide) & (df_pos.loc[:,s_color])].index
            print(len(ls_index))
            ax.scatter(data=df_scene.loc[ls_index],x='DAPI_X',y='DAPI_Y',label=f'{s_color}',s=2,color=colors[idxs])
        #break
    
    ax.set_title(f"{s_slide} {df_norm.loc[df_norm.slide_scene==s_slide,'Tissue'].iloc[0]}", fontsize=16) # \n {d_a[s_slide]}
    ax.axis('equal')
    ax.set_ylim(ax.get_ylim()[::-1])
    #ax.set_xticklabels('')
    #ax.set_yticklabels('')
    #break
    plt.legend(markerscale=3,framealpha=.5,bbox_to_anchor=(1,1)) 
    #fig.savefig(f'{codedir}/{s_date}/{s_slide}_leiden{resolution}_scatterplot.png')
    break

In [ ]:
ls_merge = tum_clust# fb_clust #tum_clust #['0','12'] #16
adata.obs.leiden = df_p.leiden
adata.obs['test'] = (adata.obs.leiden.isin(ls_merge)).replace({True:1,False:0})
sc.pl.umap(adata, color='test',title='tumor')
print(sum(adata.obs['test'])/len(adata))

In [ ]:
adata.obs['leiden'] = df_p['leiden']
fig,ax = plt.subplots(figsize=(3,3),dpi=200)
figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}_{s_type}.png'
sc.pl.umap(adata, color='leiden',ax=ax,save=figname,title=figname.split('.png')[0].replace('_',' '))

In [ ]:
# for s_merge in tum_clust:
#     ls_merge = [s_merge]
#     adata.obs['test'] = (adata.obs.leiden.isin(ls_merge)).replace({True:1,False:0})
#     sc.pl.umap(adata, color='test',title=s_merge)

## add celltypes

In [ ]:
df_p.loc[df_p.leiden.isin(tum_clust),'leidencelltype3'] = 'epithelial'
df_p.loc[df_p.leiden.isin(str_clust),'leidencelltype3'] = 'stromal'
df_p.loc[df_p.leiden.isin(fb_clust),'leidencelltype3'] = 'stromal'
df_p.loc[df_p.leiden.isin(endo_clust),'leidencelltype3'] = 'stromal'
df_p.loc[df_p.leiden.isin(imm_clust),'leidencelltype3'] = 'immune'
adata.obs['leidencelltype3'] = df_p.leidencelltype3
fig,ax = plt.subplots(figsize=(3,2),dpi=200)
figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}_{s_type}leidencelltype3.png'
sc.pl.umap(adata, color='leidencelltype3',ax=ax,save=figname,title=figname.split('.png')[0].replace('_',' '),legend_loc='on data')

In [ ]:
df_p.loc[df_p.leiden.isin(tum_clust),'leidencelltype5'] = 'epithelial'
df_p.loc[df_p.leiden.isin(str_clust),'leidencelltype5'] = 'stromal'
df_p.loc[df_p.leiden.isin(fb_clust),'leidencelltype5'] = 'fibroblast'
df_p.loc[df_p.leiden.isin(endo_clust),'leidencelltype5'] = 'endothelial'
df_p.loc[df_p.leiden.isin(imm_clust),'leidencelltype5'] = 'immune'
adata.obs['leidencelltype5'] = df_p.leidencelltype5
fig,ax = plt.subplots(figsize=(3,2),dpi=200)
figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}_{s_type}_leidencelltype5.png'
sc.pl.umap(adata, color='leidencelltype5',ax=ax,save=figname,title=figname.split('.png')[0].replace('_',' '))

In [ ]:
#visulaize
ls_col = ['CD31','ECAD','Vimentin','COL1A1','CD45']
figname = f"Umap_markers_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
title=figname.split('.png')[0].replace('_',' ')
sc.pl.umap(adata, color=ls_col,vmin='p1.5',vmax='p99',ncols=5,save=figname)

In [ ]:
#df_p.groupby('slide').leidencelltype5.value_counts(normalize=True).unstack().sort_values(by='epithelial')[25:40]

In [ ]:
df_p.loc[df_p.leiden.isin(tum_clust),'leidencelltype4'] = 'epithelial'
df_p.loc[df_p.leiden.isin(str_clust),'leidencelltype4'] = 'stromal'
df_p.loc[df_p.leiden.isin(fb_clust),'leidencelltype4'] = 'stromal'
df_p.loc[df_p.leiden.isin(endo_clust),'leidencelltype4'] = 'endothelial'
df_p.loc[df_p.leiden.isin(imm_clust),'leidencelltype4'] = 'immune'
adata.obs['leidencelltype4'] = df_p.leidencelltype4
fig,ax = plt.subplots(figsize=(3,2),dpi=200)
figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}_{s_type}_leidencelltype4.png'
sc.pl.umap(adata, color='leidencelltype4',ax=ax,save=figname,title=figname.split('.png')[0].replace('_',' '),legend_loc='on data')

In [ ]:
df_p['annotated'] = df_p.leiden.map(d_named).fillna('_AF')
#plot all groups spatially - annotated 
colors =  mpl.cm.tab10.colors + mpl.cm.Accent.colors #mpl.cm.tab20b.colors + mpl.cm.tab20c.colors

df_pos = analyze.celltype_to_bool(df_p,'annotated')
df_pos.rename({'T cell':'CD4 T cell'},axis=1,inplace=True)
ls_clust = sorted(df_pos.columns.tolist()[1::])#['8','11','12','7','5','10','1','9']#'0','2',
for s_slide in sorted(set(df_p.slide_scene)):
    s_slide = 'U54-TMA-9_sceneG06'
    fig,ax = plt.subplots(figsize=(3,3),dpi=300)
    #plot negative cells
    df_scene = df_data[df_data.slide_scene.str.contains(s_slide)]
    ax.scatter(data=df_scene,x='DAPI_X',y='DAPI_Y',color='silver',s=1,label=f'')
    #for idxs, s_color_int in enumerate(range(len(df_pos.columns))):
    for idxs, s_color in enumerate(ls_clust): #enumerate(df_pos.columns):
        #s_color = s_clust
        s_color = str(s_color)
        if len(df_p[(df_p.slide_scene==s_slide) & (df_pos.loc[:,s_color])])>=1:
            #plot positive cells
            ls_index = df_p[(df_p.slide_scene==s_slide) & (df_pos.loc[:,s_color])].index
            ax.scatter(data=df_data.loc[ls_index.astype('int')],x='DAPI_X',y='DAPI_Y',label=f'{s_color}',s=2,color=colors[idxs])
        #break
    ax.set_title(f"{s_slide}", fontsize=12) # \n {d_a[s_slide]}
    ax.axis('equal')
    #ax.set_ylim(ax.get_ylim()[::-1])
    #ax.set_xticklabels('')
    #ax.set_yticklabels('')
    #break
    plt.legend(markerscale=5,framealpha=.5,bbox_to_anchor=(1,1),title='Cell types') 
    fig.savefig(f'{codedir}/{s_date}/{s_slide}_leiden{resolution}_scatterplot.png')
    break

In [ ]:
#save
s_out = f'{s_sample}_LeidenClustering_neighbors{n_neighbors}_resolution{resolution}_markers{n_markers}_{s_type}.csv'
df_out = df_p
if  os.path.exists(s_out):
    print(f'saving csv {s_out}')
    df_out.to_csv(s_out)

In [ ]:
#s_out = '20240523_Her2B-K154-157_s_LeidenClustering_neighbors30_resolution0.6_markers25_Her2B-K157-6S.csv'
#s_out = '20240523_Her2B-K154-157_s_LeidenClustering_neighbors30_resolution0.6_markers24_Her2B-K154-6S.csv'
s_out

## Summarize Tissue Variables <a name="tissue"></a>

per patient means


[contents](#contents)

In [ ]:
os.chdir(datadir)
#s_out = '20240523_Her2B-K154-157_s_LeidenClustering_neighbors30_resolution0.6_markers25_Her2B-K157-6S.csv'
#s_out = '20240523_Her2B-K154-157_s_LeidenClustering_neighbors30_resolution0.6_markers24_Her2B-K154-6S.csv'

In [ ]:
s_sample ='20240528_U54-TMA-Normal-MIBI_s'
#leiden cell types (fraction)
n_neighbors=30
resolution= 0.6

s_type= 'MIBI'#
s_subtype = s_type
if s_type == 'MIBI':
    n_markers=25
else:
    n_markers=24

df_lei = pd.read_csv(f'{s_sample}_LeidenClustering_neighbors{n_neighbors}_resolution{resolution}_markers{n_markers}_{s_type}.csv',index_col=0)
#df_lei['slide_scene'] = [item.split('_cell')[0] for item in df_lei.index]
df_lei['leidencelltype2'] = df_lei.leidencelltype3.replace({'tumor':'epithelial','endothelial':'stromal','immune':'stromal'})
df_lei['celltype1'] = 'all'
df_lei['countme'] = True
df_lei['matched']=df_lei.leidencelltype4.copy()
df_lei.loc[df_lei.annotated.str.contains('B cell'),'matched'] = 'B cell'
df_lei.loc[df_lei.annotated.str.contains('T cell'),'matched'] = 'T cell'
df_lei.loc[df_lei.annotated == 'panCK+ ep.','matched'] = 'Ductal'
#leiden
for s_celltype in ['celltype1']: #'celltype3','celltype','leidencelltype2',
    for s_cell in df_lei.loc[:,s_celltype].unique():
        for s_grouper in ['slide_scene','celltype1']:
            df_cell = df_lei.loc[df_lei.loc[:,s_celltype]==s_cell]
            df_prop = viz.prop_positive(df_cell,s_cell='matched',s_grouper=s_grouper)
            s_out = f'results_{s_sample}_LeidenClustering_{n_neighbors}_{n_markers}_{resolution}_by{s_grouper}_by{s_celltype}_in{s_cell}_{s_type}.csv'
            df_prop.fillna(0).to_csv(s_out)
            print(s_out)

## compare tumor, border and normal

done in cycif notebook